### Running StyleGAN-2 Model by importing pretrained weights
Need tesnsorflow v1, GPU and a lot of free space in Google Drive

In [ ]:
%tensorflow_version 1.x

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/NVlabs/stylegan.git
%cd stylegan

In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config
 
def main():
    # Initialize TensorFlow.
    tflib.init_tf()
 
    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # The link can be found in the official repo
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.
 
    # Print network details.
    Gs.print_layers()
 
    # Pick latent vector.
    rnd = np.random.RandomState(10)  # seed = 10
    latents0 = rnd.randn(1, Gs.input_shape[1])
    latents1 = rnd.randn(1, Gs.input_shape[1])
    latents2 = rnd.randn(1, Gs.input_shape[1])
    latents3 = rnd.randn(1, Gs.input_shape[1])
    latents4 = rnd.randn(1, Gs.input_shape[1])
    latents5 = rnd.randn(1, Gs.input_shape[1])
    latents6 = rnd.randn(1, Gs.input_shape[1])
    
    num_split = 39  
    for i in range(40):
        latents = latents6+(latents0-latents6)*i/num_split
        # Generate image.
        fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)
 
        # Save image.
        os.makedirs(config.result_dir, exist_ok=True)
        png_filename = os.path.join(config.result_dir, 'photo'+'{0:04d}'.format(i)+'.png')
        PIL.Image.fromarray(images[0], 'RGB').save(png_filename)
 
if __name__ == "__main__":
    main()

In [ ]:
from PIL import Image
import glob
 
files = sorted(glob.glob('results/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('stylegan.gif', save_all=True, 
               append_images=images[1:], 
               duration=200, loop=0)